In [1]:
# Code for running out-of-the-box tinyLlama on climateBUG dataset

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
import torch
import json
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from preprocessing import climateBUG_reduce_rows
from sklearn.metrics import accuracy_score, f1_score

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.current_device()

0

In [5]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",)

model =  AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0",
                                              torch_dtype=torch.float16,
                                              device_map="auto",
                                              low_cpu_mem_usage=True
    ).to("cuda")

/home/lukeg/ClimatEnv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
df = pd.read_json("/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/data/climateBUG/climateBUG-testing-dataset.json")
preprocessed_testing_df = climateBUG_reduce_rows(df, rows=1000)

                                             statement  label
0    4CRÉDIT AGRICOLE S.A. – 2020-2021 INTEGRATED R...      0
1    p.11Exceptional financial  strength .............      0
2    p.18Our strategy: be the global  relationship ...      0
3    p.30Creatingadded valueOur business model serv...      0
4    p.40Committing and upholding  our responsibili...      0
..                                                 ...    ...
995  In  the  future,  Societe  Generale  is  commi...      1
996                                       !      0
997                                  !' &\t)      0
998  ):A@\t6/,"+,2+$)...      1
999  The  Hydrogen  Council  nowbrings  together  m...      1

[1000 rows x 2 columns]
                                             statement  label
0    4CRÉDIT AGRICOLE S.A. – 2020-2021 INTEGRATED R...      0
1    p.11Exceptional financial  strength .............      0
2    p.18Our strategy: be the global  relationship ... 

In [7]:
testing_dataset = Dataset.from_pandas(preprocessed_testing_df)

In [8]:
# Because TinyLlama is a generative model, it works very differently to the BERT-based models.
# We therefore get it to generate a label, and prompt it like any chatbot
# Return tensors mean we get pytorch tensors instead of python lists

def classify_text(text):
    prompt = f"""Determine if the following sentence is RELATED (1) or UNRELATED (2) to climate change.
    
    Sentence: {text}
    The classification is:"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=2
    )

    return tokenizer.decode(
        outputs[0][-2],
        skip_special_tokens=True
    ).strip()

In [9]:
# Zero shot check

print(classify_text(
    "Let's talk about sustainability"
))

Rel


In [10]:
def custom_tokenize(examples):
    tokenized_output = tokenizer(
        text=examples["statement"],
        max_length=512, 
        padding="max_length", 
        truncation=True)

    return tokenized_output

tokenized_testing_dataset = testing_dataset.map(custom_tokenize, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [11]:
def calculate_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")

    return {
        "accuracy": accuracy,
        "f1_score": f1,
    }

In [12]:
training_args  = TrainingArguments(
    output_dir="/home/lukeg/Documents/VS_code/fine_tuning/lxg406/climate_relatedness_classification/vanilla/results/tinyLlama/first_run",
    do_train=False,
    do_eval=True,
    )

In [17]:
model.gradient_checkpointing_enable()

In [18]:
trainer  = Trainer(
    model=model,
    args= training_args,
    tokenizer=tokenizer,
    eval_dataset=tokenized_testing_dataset,
    compute_metrics=calculate_metrics,
)

In [19]:
metrics = trainer.evaluate()

OutOfMemoryError: CUDA out of memory. Tried to allocate 500.00 MiB. GPU 0 has a total capacity of 3.69 GiB of which 27.75 MiB is free. Including non-PyTorch memory, this process has 3.64 GiB memory in use. Of the allocated memory 3.49 GiB is allocated by PyTorch, and 60.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
with open("./results/tinyLlama/climateBUG/first_run/eval_metrics.json", "w") as output_file:
    json.dump(metrics, output_file)